<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/Audiovisual_basic_TFServing_on_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/'

dataset_train_0-20.npy


In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/dataset_train_0-20.npy' "dataset_train_0-20.npy"

In [4]:
!ls

dataset_train_0-20.npy	gdrive	sample_data


In [5]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print(tf.__version__)

1.14.0


In [0]:
dataset_train = np.load("dataset_train_0-20.npy")

In [0]:
# Split dataset into inputs and ground truths
x_train = dataset_train[:,:,:,:,0 ]
y_train = dataset_train[:,:,:,:,1:]

In [8]:
print(x_train.shape)
print(y_train.shape)

(210, 298, 257, 2)
(210, 298, 257, 2, 2)


In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab TFServing/saved_model_basic_22_09_045241.h5' "saved_model_basic_22_09_045241.h5"

In [10]:
from tensorflow.keras.models import load_model
model = load_model("saved_model_basic_22_09_045241.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Most of the following code in this notebook was adapted from https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple

In [11]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/1/saved_model.pb

Saved model:
total 604
-rw-r--r-- 1 root root 613190 Sep 26 00:26 saved_model.pb
drwxr-xr-x 2 root root   4096 Sep 26 00:26 variables


In [12]:
'''
MODEL_DIR = ""
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('Already saved a model, cleaning up\n')
  !rm -r {export_path}

tf.contrib.saved_model.save_keras_model(model, export_path)

#!mkdir {export_path}
!ls
'''

'\nMODEL_DIR = ""\nversion = 1\nexport_path = os.path.join(MODEL_DIR, str(version))\n\nprint(\'export_path = {}\n\'.format(export_path))\nif os.path.isdir(export_path):\n  print(\'Already saved a model, cleaning up\n\')\n  !rm -r {export_path}\n\ntf.contrib.saved_model.save_keras_model(model, export_path)\n\n#!mkdir {export_path}\n!ls\n'

In [13]:
print('\nSaved model:')
!ls -l {export_path}


Saved model:
total 604
-rw-r--r-- 1 root root 613190 Sep 26 00:26 saved_model.pb
drwxr-xr-x 2 root root   4096 Sep 26 00:26 variables


In [14]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 298, 257, 2)
        name: input_layer_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_layer/Reshape:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 298, 257, 2, 2)
        name: output_layer/Reshape:0
  Method name is: tensorflow/serving/predict


In [15]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  20296      0 --:--:-- --:--:-- --:--:-- 20296
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.

In [16]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 125 not upgraded.
Need to get 151 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 1.14.0 [151 MB]
Fetched 151 MB in 2s (71.3 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_1.14.0_all.deb ...
Unpacking tensorflow-model-server (1.14.0) ...
Setting up tensorflow-model-server (1.14.0) ...


In [0]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [18]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=avspeech_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [19]:
!tail server.log

2019-09-26 00:27:32.043348: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2019-09-26 00:27:32.051945: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-09-26 00:27:32.095588: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:202] Restoring SavedModel bundle.
2019-09-26 00:27:32.198176: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:311] SavedModel load for tags { serve }; Status: success. Took 162548 microseconds.
2019-09-26 00:27:32.198316: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:103] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2019-09-26 00:27:32.198420: I tensorflow_serving/core/loader_harness.cc:86] Successfully loaded servable version {name: avspeech_model version: 1}
2019-09-26 00:27:32.201127: I tensorflow_serving/mo

In [20]:
'''
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))
'''

"\ndef show(idx, title):\n  plt.figure()\n  plt.imshow(test_images[idx].reshape(28,28))\n  plt.axis('off')\n  plt.title('\n\n{}'.format(title), fontdict={'size': 16})\n\nimport random\nrando = random.randint(0,len(test_images)-1)\nshow(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))\n"

In [0]:
!pip install -q requests

In [0]:
test_images = x_train

In [23]:
test_images[0:1].shape

(1, 298, 257, 2)

In [24]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:1].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ...  0.28270596265792847], [0.2902107834815979, 0.0]]]]}


In [0]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=data, headers=headers)

In [0]:
predictions = json.loads(json_response.text)['predictions']

In [27]:
print("result string length:", len(json_response.text))
print(json_response.text[:128])

result string length: 4371305
{
    "predictions": [[[[[0.50045526, 0.499316454], [0.49955678, 0.499397397]], [[0.500694752, 0.499889225], [0.499602556, 0.499


In [28]:
print(len(predictions))
print(str(predictions)[:100])
#print(predictions)

1
[[[[[0.50045526, 0.499316454], [0.49955678, 0.499397397]], [[0.500694752, 0.499889225], [0.499602556


In [29]:
import ast
results = ast.literal_eval(str(predictions))
results = np.array(results)
print(results.shape)
print(results[0,:1,:3])

(1, 298, 257, 2, 2)
[[[[0.50045526 0.49931645]
   [0.49955678 0.4993974 ]]

  [[0.50069475 0.49988922]
   [0.49960256 0.49962071]]

  [[0.50083208 0.49971414]
   [0.49978095 0.49952585]]]]


In [0]:
np.save("avspeech_output", results)

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp "avspeech_output.npy" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/avspeech_output.npy"

In [0]:
# Call a particular version
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [0]:
import time

In [35]:
headers = {"content-type": "application/json"}
start_time = time.time()
json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=data, headers=headers)
end_time = time.time()
print(end_time-start_time)

10.193274736404419


In [36]:
headers = {"content-type": "application/json"}

times = []
num_tests = 30

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Number of test: 30
Test times: [10.216963291168213, 10.204941034317017, 10.252147197723389, 10.293035745620728, 10.263291835784912, 9.966241836547852, 10.026309490203857, 10.164400577545166, 10.15079379081726, 10.0349862575531, 9.99442458152771, 9.909123182296753, 10.189841985702515, 10.261853456497192, 10.364156484603882, 10.088849306106567, 9.945067882537842, 10.001926183700562, 10.098788976669312, 10.057594060897827, 9.863841533660889, 10.099037647247314, 10.213852882385254, 10.159438371658325, 9.990747928619385, 10.093352317810059, 10.125235557556152, 10.038752555847168, 9.996816396713257, 10.06305480003357]
Average running time: 10.104295571645102


In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 30

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 100

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

In [0]:
headers = {"content-type": "application/json"}

times = []
num_tests = 100

for i in range(num_tests):
  d = json.dumps({"signature_name": "serving_default", "instances": test_images[i:i+1].tolist()})
  start_time = time.time()
  json_response = requests.post('http://localhost:8501/v1/models/avspeech_model:predict', data=d, headers=headers)
  end_time = time.time()
  runtime = end_time-start_time
  times.append(runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

In [0]:
np.savetxt("TFServing_CPU_times.csv", times, delimiter=",")

In [0]:
!cp "TFServing_CPU_times.csv" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_basic_output/TFServing_CPU_times.csv"